# Data Analysis

TO BE SUPPLIED


## Shapely - Converting JSON to geometry objects
Fiona will be treated in a next lesson. Here we mainly use Fiona 
to read Vector data (Features) into memory for subsequent Shapely manipulation.

Feature geometry can be accessed using the `geometry` property of each feature, for example
we can open the dataset that contains a (Multi)Polygon for each country and print
out the geometry of the 10th Feature:

First we import `Shapely` and its functions and then convert the JSON-encoded geometries to Geometry objects
using the `shape` function.

In [ ]:
import fiona
from shapely.geometry import shape

with fiona.open("../data/countries.3857.gpkg") as countries:
	country = countries[4]
	print("This is %s" % country["properties"]["NAME"])
	geom = shape(country["geometry"])
  
geom # Jupyter can display geometry data directly

In [ ]:
print(geom.type)

In [ ]:
print(geom.area)

In [ ]:
# In km
print(geom.length/1000)

Let's have a look at some geometry methods. 
Tip: Shapely code is well-documented, you can always use the Python built-in `help()` function.

In [ ]:
help(geom)

For example we can make a buffer of 500 meter around our polygon (making Canada somewhat bigger):

In [ ]:
buffered_geom = geom.buffer(500)
buffered_geom

In [ ]:
# In km
buffered_geom.length/1000

### Converting the geometry back to JSON format
Once we are finished, we can convert the geometry back to JSON format using `shapely.geometry.mapping` function


In [ ]:
from shapely.geometry import mapping

In [ ]:
# let's create new GeoJSON-encoded vector feature

new_feature = {
	"type": "Feature",
	"properties": {"name": "My buffered feature"},
	"geometry": mapping(buffered_geom)
}
new_feature

# Now we could e.g. write the Feature back to file


## Lightweight Analysis with Shapely
Let's explore some of Shapely's functions for *(Topological) Spatial Relationships*.
See simplified view below, more [on Wikipedia](https://en.wikipedia.org/wiki/Spatial_relation).

![(Topological) Spatial Relationships](images/spatialrelations.png)
*Examples of topological spatial relations - [By Krauss - Own work, CC BY-SA 3.0](https://commons.wikimedia.org/w/index.php?curid=21299138)*
 

If you want to dive into
a theoretical background, check out the [Dimensionally Extended nine-Intersection Model (DE-9IM)](https://en.wikipedia.org/wiki/DE-9IM). 

We'll start simple, answering the question: **Does the Donau float through Romenia?**. We'll use two National Earth datasets: Rivers and lakes
(LineStrings) and (Admin-0) Countries. In spatial 
relationship-terms: *Is the (Multi)LineString representing the Donau crossing the (Multi)Polygon representing Romenia?"* 

We will follow these steps:

* read the Rivers and Lakes dataset
* extract the Donau feature Geometry  
* read the Countries dataset
* extract the Romania feature Geometry  
* apply the Shapely `crosses` function
* advanced: list all DE-9IM relations of the two geometries
* advanced: through which countries does the Donau float?


In [ ]:
import fiona
from shapely.geometry import shape


### Fetch the Donau geometry.


In [ ]:
with fiona.open("../data/rivers_lake_centerlines.gpkg") as rivers_lakes:
	for feature in rivers_lakes:
		if feature['properties']['name'] == 'Donau':
			donau_geom = shape(feature["geometry"])
			print('Found Donau! geom type: %s' % donau_geom.type)
			break


### Fetch the Romania geometry.


In [ ]:
with fiona.open("../data/countries.json") as countries:
	for feature in countries:
		if feature["properties"]["NAME"] == "Romania":
			romania_geom = shape(feature["geometry"])
			print('Found Romania! geom type: %s' % romania_geom.type)
			break


### Does the Donau float through Romenia?


In [ ]:
donau_geom.crosses(romania_geom)


### List the (DE-9IM) relationships.


In [ ]:
donau_geom.relate(romania_geom)


### Through which countries does the Donau float?


In [ ]:
print("The Donau floats through:")

with fiona.open("../data/countries.gpkg") as countries:
	for feature in countries:
		country_geom = shape(feature["geometry"])
		if donau_geom.crosses(country_geom):
			print(feature["properties"]["NAME"])
  

---
[<- Raster Data](05-raster-data.ipynb) | [Visualization ->](07-visualization.ipynb)
